## Extract finance digit from 10-k

In [1]:
import os
import re
import pandas as pd

In [2]:
path = '/Users/faustune/PycharmProjects/599project/data1/'

In [3]:
folds = os.listdir(path)
folds

['.DS_Store', '1996.full']

In [18]:
df = pd.DataFrame(columns = ['filename','<PERIOD-START>','<PERIOD-END>','<SEC-DOCUMENT>','<SEC-HEADER>','ACCESSION NUMBER:','PUBLIC DOCUMENT COUNT:',
                                 'CONFORMED PERIOD OF REPORT:','FILED AS OF DATE:','COMPANY CONFORMED NAME:',
                                 'CENTRAL INDEX KEY:','STANDARD INDUSTRIAL CLASSIFICATION:','IRS NUMBER:',
                                 'STATE OF INCORPORATION:','FISCAL YEAR END:','SEC FILE NUMBER:','FILM NUMBER:',
                                 'CITY:','STATE:', 'Total assets', "Total shareholders' equity", 
                             "Total stockholders' equity",'Total liabilities',  'Provision for income taxes',
                                'Long-term debt    ', 'Operating income', 'Gross profit', 'Net income',  
                               '<TOTAL-COSTS>', '<TOTAL-REVENUES>', '<RECEIVABLES>', '<INTEREST-EXPENSE>', 
                               '<CURRENT-ASSETS>', '<CURRENT-LIABILITIES>', '<DEPRECIATION>']) 
df

,filename,<PERIOD-START>,<PERIOD-END>,<SEC-DOCUMENT>,<SEC-HEADER>,ACCESSION NUMBER:,PUBLIC DOCUMENT COUNT:,CONFORMED PERIOD OF REPORT:,FILED AS OF DATE:,COMPANY CONFORMED NAME:,...,Operating income,Gross profit,Net income,<TOTAL-COSTS>,<TOTAL-REVENUES>,<RECEIVABLES>,<INTEREST-EXPENSE>,<CURRENT-ASSETS>,<CURRENT-LIABILITIES>,<DEPRECIATION>


In [19]:
for fold in folds:
    if fold != '.DS_Store':
        files = os.listdir(path+fold)
        for filename in files:
            if filename != '.DS_Store':
                with open(path+fold+'/'+filename,'r') as file:
                    flag_item8 = 0
                    flag_cur_asset = 0
                    lines = file.readlines()
                    dic = {}
                    dic['filename'] = filename
                    item1 = ['<SEC-DOCUMENT>','<SEC-HEADER>','ACCESSION NUMBER:','PUBLIC DOCUMENT COUNT:',
                                 'CONFORMED PERIOD OF REPORT:','FILED AS OF DATE:','COMPANY CONFORMED NAME:',
                                 'CENTRAL INDEX KEY:','STANDARD INDUSTRIAL CLASSIFICATION:','IRS NUMBER:',
                                 'STATE OF INCORPORATION:','FISCAL YEAR END:','SEC FILE NUMBER:','FILM NUMBER:',
                                 'CITY:','STATE:']
                    item8_1 = ['Total liabilities',  'Provision for income taxes',
                                'Long-term debt    ']
                    item8_2 = ['Total assets', "Total shareholders' equity", "Total stockholders' equity",
                               'Operating income', 'Gross profit', 'Net income', '<PERIOD-START>' ,
                               '<TOTAL-COSTS>', '<TOTAL-REVENUES>', '<RECEIVABLES>', '<INTEREST-EXPENSE>', 
                               '<CURRENT-ASSETS>', '<CURRENT-LIABILITIES>', '<DEPRECIATION>', '<PERIOD-END>']
                    for line in lines:
                        if flag_cur_asset == 1:
                            if re.search('Cash', line, re.IGNORECASE) != None:
                                if re.search('\d',line) != None:
                                    start = line.index(re.search('\d',line).group(0))
                                    dic['Cash'] = line[start:].strip()
                            flag_cur_asset = 0
                        for key in item1:
                            if re.search(key, line) != None:
                                dic[key] = line[line.index(':')+1:].strip()
                        if re.search('ITEM', line) != None:
                            if re.search('8. ', line) != None:
                                if re.search('FINANCIAL STATEMENTS', line) != None:
                                    flag_item8 = 1
                        if flag_item8 == 1:
                            for key in item8_1:
                                if re.search(key, line, re.IGNORECASE) != None:
                                    if re.search('\d',line) != None:
                                        start = line.index(re.search('\d',line).group(0))
                                    if re.search(' ',line[start:]) != None:
                                        end = line[start:].index(re.search(' ',line[start:]).group(0))
                                        dic[key] = line[start:start+end+1]
                                    item8_1.remove(key)
                            for key in item8_2:
                                if re.search(key, line, re.IGNORECASE) != None:
                                    if re.search('\d',line) != None:
                                        start = line.index(re.search('\d',line).group(0))
                                        dic[key] = line[start:].strip()
                                    item8_2.remove(key)
                        if re.search('Current assets', line, re.IGNORECASE) != None:
                            flag_cur_asset = 1
                df = df.append(dic,ignore_index=True)              
#                 print(dic,len(dic))
#                                 
# Total assets   Total liabilities 

In [20]:
df

,filename,<PERIOD-START>,<PERIOD-END>,<SEC-DOCUMENT>,<SEC-HEADER>,ACCESSION NUMBER:,PUBLIC DOCUMENT COUNT:,CONFORMED PERIOD OF REPORT:,FILED AS OF DATE:,COMPANY CONFORMED NAME:,...,Gross profit,Net income,<TOTAL-COSTS>,<TOTAL-REVENUES>,<RECEIVABLES>,<INTEREST-EXPENSE>,<CURRENT-ASSETS>,<CURRENT-LIABILITIES>,<DEPRECIATION>,Cash
0,00087B101-10-K-19960529.txt,01-1995,29-1996,19960530,19960530,0000912057-96-011031,5,19960229,19960529,ACR GROUP INC,...,"10,720,806 8,563,240 6,738,307","183,766 $ 558,206 $ 1,114,068","45,779,447",0,"7,648,340","644,767","17,758,665","9,640,604","1,470,788","348,162 $ 162,745"


In [17]:
df.to_csv('data.csv', index = False)

In [ ]:
# We focused our work on items 8 and 14, which contain audited balance sheets for two years as well as 
# three audited annual statements of income and cash flow, and supplemental financial data schedules (FDS). 
# The FDS consist of aggregated financial information and selected financial ratios.